In [5]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from collections import Counter


# CARGA DE DATOS
train_data = pd.read_csv("../Data/train_indexado.csv")
valid_data = pd.read_csv("../Data/valid_indexado.csv")
test_data = pd.read_csv("../Data/test_indexado.csv")

# CARGA DEL MAPEADO EKMAN
with open("../Data/GoEmotions/ekman_mapping.json", "r") as f:
    ekman_mapping = json.load(f)

# Asegurar inclusión de 'neutral'
ekman_mapping["neutral"] = ["neutral"]
reverse_ekman_map = {e: cat for cat, emos in ekman_mapping.items() for e in emos}
ekman_labels = list(ekman_mapping.keys())
ekman_to_id = {label: idx for idx, label in enumerate(ekman_labels)}
id_to_ekman = {idx: label for label, idx in ekman_to_id.items()}

# IDENTIFICAR EMOCIONES ORIGINALES (GoEmotions)
emotion_columns = [col for col in train_data.columns if col not in ["Text", "ID"]]


# FUNCIÓN PARA OBTENER EMOCIONES EKMAN POR FILA
def get_ekman_emotions(row):
    active_ekman = set()
    for emo in emotion_columns:
        if row[emo] == 1 and emo in reverse_ekman_map:
            active_ekman.add(reverse_ekman_map[emo])
    return list(active_ekman)


# Función para graficar distribuciones
def plot_ekman_distribution(ekman_lists, title, filename, color):
    flat_ekmans = [e for sublist in ekman_lists for e in sublist]
    counts = Counter(flat_ekmans)

    plt.figure(figsize=(10, 5))
    bars = plt.bar(counts.keys(), counts.values(), color=color)
    plt.title(title)
    plt.ylabel("Número de registros")
    plt.xticks(rotation=45)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(height),
                 ha='center', va='bottom', fontsize=9)
        
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()



def plot_ekman_distribution_after(ekman_ids, title, filename, color):
    labels = [id_to_ekman[i] for i in ekman_ids]
    counts = Counter(labels)

    plt.figure(figsize=(10, 5))
    bars = plt.bar(counts.keys(), counts.values(), color=color)
    plt.title(title)
    plt.ylabel("Número de registros")
    plt.xticks(rotation=45)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(height),
                 ha='center', va='bottom', fontsize=9)
        
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()



def plot_distribution(solo_una, conflictivos, title, filename, color):
    labels = ["Una emoción de Ekman", "Múltiples emociones de Ekman"]
    values = [solo_una, conflictivos]

    plt.figure(figsize=(7, 5))
    bars = plt.bar(labels, values, color=color)
    plt.title(title)
    plt.ylabel("Número de registros")

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(height),
                 ha='center', va='bottom', fontsize=9)
        
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# APLICAR FUNCIÓN 
train_data["Ekman"] = train_data.apply(get_ekman_emotions, axis=1)
valid_data["Ekman"] = valid_data.apply(get_ekman_emotions, axis=1)
test_data["Ekman"]  = test_data.apply(get_ekman_emotions, axis=1)


# GENERAR GRÁFICOS DE DISTRIBUCIÓN DE EMOCIONES EKMAN
plot_ekman_distribution(train_data["Ekman"], "Train: Based On Ekman (antes)", "../Plots/Experiment1/Ekman/train_ekman_before.png", "skyblue")
plot_ekman_distribution(valid_data["Ekman"], "Valid: Based On Ekman (antes)", "../Plots/Experiment1/Ekman/valid_ekman_before.png", "orange")
plot_ekman_distribution(test_data["Ekman"],  "Test: Based On Ekman (antes)",  "../Plots/Experiment1/Ekman/test_ekman_before.png", "lightgreen")

# FUNCIÓN PARA OBTENER ÍNDICES CONFLICTIVOS
def get_conflicting_indices(df):
    return [i for i, x in enumerate(df["Ekman"]) if len(x) > 1]

# OBTENER Y CONTAR REGISTROS CONFLICTIVOS
conflict_train_idx = get_conflicting_indices(train_data)
conflict_valid_idx = get_conflicting_indices(valid_data)
conflict_test_idx  = get_conflicting_indices(test_data)

solo_una_train = len(train_data) - len(conflict_train_idx)
solo_una_valid = len(valid_data) - len(conflict_valid_idx)
solo_una_test  = len(test_data) - len(conflict_test_idx)



plot_distribution(solo_una_train, len(conflict_train_idx), 
                  "Train: Un sentimiento vs. Más de uno", 
                  "../Plots/Experiment1/Ekman/Distribution/distribucion_train.png", 
                  "lightcoral")

plot_distribution(solo_una_valid, len(conflict_valid_idx), 
                  "Valid: Un sentimiento vs. Más de uno", 
                  "../Plots/Experiment1/Ekman/Distribution/distribucion_valid.png", 
                  "lightsalmon")

plot_distribution(solo_una_test, len(conflict_test_idx), 
                  "Test: Un sentimiento vs. Más de uno", 
                  "../Plots/Experiment1/Ekman/Distribution/distribucion_test.png", 
                  "lightseagreen")


print("\nCONTEO DE REGISTROS POR CATEGORÍAS DE SENTIMIENTO:")
print("Train - Solo una categoría:", solo_una_train, "| Más de una:", len(conflict_train_idx))
print("Valid - Solo una categoría:", solo_una_valid, "| Más de una:", len(conflict_valid_idx))
print("Test  - Solo una categoría:", solo_una_test,  "| Más de una:", len(conflict_test_idx))


CONTEO DE REGISTROS POR CATEGORÍAS DE SENTIMIENTO:
Train - Solo una categoría: 39555 | Más de una: 3855
Valid - Solo una categoría: 4946 | Más de una: 480
Test  - Solo una categoría: 4968 | Más de una: 459


In [6]:
# ELIMINAR REGISTROS CON MÁS DE UNA CATEGORÍA EKMAN
train_data_clean = train_data.drop(index=conflict_train_idx).reset_index(drop=True)
valid_data_clean = valid_data.drop(index=conflict_valid_idx).reset_index(drop=True)
test_data_clean  = test_data.drop(index=conflict_test_idx).reset_index(drop=True)


ekman_to_id = {label: idx for idx, label in enumerate(ekman_labels)}

# Categorización usando Ekman
def categorize_emotions(row):
    for emotion in row['Ekman']:
        if emotion in reverse_ekman_map:
            ekman_category = reverse_ekman_map[emotion]
            return ekman_to_id[ekman_category]
    return len(ekman_labels)  # "otros" si no encuentra

train_data_clean['Ekman'] = train_data_clean.apply(categorize_emotions, axis=1)
valid_data_clean['Ekman'] = valid_data_clean.apply(categorize_emotions, axis=1)
test_data_clean['Ekman']  = test_data_clean.apply(categorize_emotions, axis=1)

# Graficar distribuciones después de la limpieza
plot_ekman_distribution_after(train_data_clean["Ekman"], "Train: Based On Ekman (después)", "../Plots/Experiment1/Ekman/train_ekman_after.png", "skyblue")
plot_ekman_distribution_after(valid_data_clean["Ekman"], "Valid: Based On Ekman (después)", "../Plots/Experiment1/Ekman/valid_ekman_after.png", "orange")
plot_ekman_distribution_after(test_data_clean["Ekman"],  "Test: Based On Ekman (después)",  "../Plots/Experiment1/Ekman/test_ekman_after.png", "lightgreen")

#Cambio de nombre de la columa Ekman a Emotion
train_data_clean.rename(columns={"Ekman": "Emotion"}, inplace=True)
valid_data_clean.rename(columns={"Ekman": "Emotion"}, inplace=True)
test_data_clean.rename(columns={"Ekman": "Emotion"}, inplace=True)

# Guardar resultados
train_data_clean[['Text', 'Emotion', 'ID']].to_csv('../Data/BasedOnEkman/train_ekman.csv', index=False)
valid_data_clean[['Text', 'Emotion', 'ID']].to_csv('../Data/BasedOnEkman/valid_ekman.csv', index=False)
test_data_clean[['Text', 'Emotion', 'ID']].to_csv('../Data/BasedOnEkman/test_ekman.csv', index=False)

print("\nArchivos guardados correctamente.")


Archivos guardados correctamente.


In [7]:
# Aplica TF-IDF sin sobreescribir train_data_clean
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Aplicar el vectorizador
sentences_train = train_data_clean['Text'].values

temp = TfidfVectorizer()

vectorizer.fit(sentences_train)

feature_names = vectorizer.get_feature_names_out()
n_features = len(feature_names)
print(f"Número de features detectados: {n_features}")




Número de features detectados: 25168
